In [ ]:
!git clone https://github.com/pytorch/fairseq.git
!cd fairseq && git checkout 9a1c497
!pip install --upgrade ./fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 31399, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31399 (delta 28), reused 39 (delta 26), pack-reused 31347
Receiving objects: 100% (31399/31399), 21.68 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (23100/23100), done.
Note: checking out '9a1c497'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 9a1c4970 Make Hydra logging work with DDP (#1568)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fairseq
  DEPRECATION: A future pip vers

In [ ]:
!pip install 'torch>=1.6.0' editdistance matplotlib 'sacrebleu==1.5.1' sacremoses sentencepiece tqdm wandb subword-nmt OpenNMT-py
!pip install --upgrade jupyter ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.2 MB/s 
     |████████████████████████████████| 880 kB 15.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.6 MB/s 
     |████████████████████████████████| 1.8 MB 38.2 MB/s 
     |████████████████████████████████| 181 kB 46.2 MB/s 
     |████████████████████████████████| 145 kB 41.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2ed1f2def8229f2b49ee4f6b1beb193ad98169296e3a5c69fcbe943d1b87aa51
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1c27717342ef925c91bc08896efe5692cccfc7e7c35f372dbbf1a204fce3b7ba
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "data.zip" -d "/content/drive/MyDrive/Notebooks/NLP/CA5/"

Archive:  data.zip
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/test.en  
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/test.fa  
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/train.en  
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/train.fa  
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/valid.en  
  inflating: /content/drive/MyDrive/Notebooks/NLP/CA5/data/valid.fa  


# Apply Basics PreProcessing

In [ ]:
src = ['train.fa','train.en','test.fa','test.en','valid.fa','valid.en']
Dst = ['train_prc.fa','train_prc.en','test_prc.fa','test_prc.en','valid_prc.fa','valid_prc.en']
for i,file in enumerate(src):
  with open('/content/drive/MyDrive/Notebooks/NLP/CA5/data/'+ file,'r') as f1:
      lines = f1.read()
  if file[-2:]=='fa':
    lines = lines.replace('\u200c',' ')
    lines = lines.replace("'", '"')
  else :
    lines = lines.lower()
    lines = lines.replace("'", '"')
  f1.close()
  with open('/content/drive/MyDrive/Notebooks/NLP/CA5/data/'+ Dst[i],'w') as f2:
    f2.write(lines)
  f2.close()

# Apply BPE

In [ ]:
!subword-nmt learn-bpe -s 25000 < drive/MyDrive/Notebooks/NLP/CA5/data/train_prc.en > drive/MyDrive/Notebooks/NLP/CA5/data/code.en

100% 25000/25000 [00:38<00:00, 652.44it/s] 


In [ ]:
!subword-nmt learn-bpe -s 20000 < drive/MyDrive/Notebooks/NLP/CA5/data/train_prc.fa > drive/MyDrive/Notebooks/NLP/CA5/data/code.fa

100% 20000/20000 [00:25<00:00, 772.08it/s]


In [ ]:
!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.en < drive/MyDrive/Notebooks/NLP/CA5/data/train_prc.en > drive/MyDrive/Notebooks/NLP/CA5/data/train-bpe.en
!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.en < drive/MyDrive/Notebooks/NLP/CA5/data/valid_prc.en > drive/MyDrive/Notebooks/NLP/CA5/data/valid-bpe.en
!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.en < drive/MyDrive/Notebooks/NLP/CA5/data/test_prc.en > drive/MyDrive/Notebooks/NLP/CA5/data/test-bpe.en

!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.fa < drive/MyDrive/Notebooks/NLP/CA5/data/train_prc.fa > drive/MyDrive/Notebooks/NLP/CA5/data/train-bpe.fa
!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.fa < drive/MyDrive/Notebooks/NLP/CA5/data/valid_prc.fa > drive/MyDrive/Notebooks/NLP/CA5/data/valid-bpe.fa
!subword-nmt apply-bpe -c drive/MyDrive/Notebooks/NLP/CA5/data/code.fa < drive/MyDrive/Notebooks/NLP/CA5/data/test_prc.fa > drive/MyDrive/Notebooks/NLP/CA5/data/test-bpe.fa

In [ ]:
!fairseq-preprocess --tokenizer space --bpe subword_nmt -s en -t fa \
  --trainpref drive/MyDrive/Notebooks/NLP/CA5/data/train-bpe \
  --validpref drive/MyDrive/Notebooks/NLP/CA5/data/valid-bpe \
  --testpref drive/MyDrive/Notebooks/NLP/CA5/data/test-bpe \
  --destdir drive/MyDrive/Notebooks/NLP/CA5/data/summary --workers 20

2022-06-05 17:32:06 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe='subword_nmt', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='drive/MyDrive/Notebooks/NLP/CA5/data/summary', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict=None, suppress_crashes=False, target_lang='fa', task='translation', tensorboard_logdir=None, testpref='drive/MyDrive/Notebooks/NLP/CA5/data/test-bpe', tgtdict=None, 

In [ ]:
!onmt_build_vocab -config en-fa.yaml -n_sample -1